In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from io import StringIO

In [2]:
service = Service(executable_path='../chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(f'https://www.investing.com/economic-calendar/')
time.sleep(2)

botton_cookies = driver.find_element(
    By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'
)
botton_cookies.click()
time.sleep(2)

for _ in range(30):
    botton_signup = driver.find_element(By.XPATH, '//*[@id="PromoteSignUpPopUp"]/div[2]/i')
    print('.', end='')
    if botton_signup.is_displayed():
        botton_signup.click()
        break
    else:
        time.sleep(2)


..

In [21]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10)

In [22]:
table = driver.find_element(
    By.XPATH, '//*[@class="eCalNew eCalMainNew ecoCalCont"]'
)
dfs = pd.read_html(StringIO(table.get_attribute('innerHTML')))
df_raw = dfs[0]
if df_raw.columns[0] is tuple: # Fix for ocasional multi-index
    df_raw.columns = [c[0] for c in df_raw.columns]
df_raw

,Time,Cur.,Imp.,Event,Actual,Forecast,Previous,Unnamed: 7,Unnamed: 8
0,"Sunday, January 1, 2023","Sunday, January 1, 2023","Sunday, January 1, 2023","Sunday, January 1, 2023","Sunday, January 1, 2023","Sunday, January 1, 2023","Sunday, January 1, 2023","Sunday, January 1, 2023","Sunday, January 1, 2023"
1,20:30,KRW,NaN,S&P Global South Korea Manufacturing PMI (Dec),48.2,NaN,49.0,NaN,NaN
2,"Monday, January 2, 2023","Monday, January 2, 2023","Monday, January 2, 2023","Monday, January 2, 2023","Monday, January 2, 2023","Monday, January 2, 2023","Monday, January 2, 2023","Monday, January 2, 2023","Monday, January 2, 2023"
3,All Day,NaN,Holiday,United States - New Year's Day,United States - New Year's Day,United States - New Year's Day,United States - New Year's Day,United States - New Year's Day,United States - New Year's Day
4,All Day,NaN,Holiday,United Kingdom - New Year's Day,United Kingdom - New Year's Day,United Kingdom - New Year's Day,United Kingdom - New Year's Day,United Kingdom - New Year's Day,United Kingdom - New Year's Day
...,...,...,...,...,...,...,...,...,...
1221,20:00,KRW,NaN,Trade Balance (Jan),-12.69B,-9.27B,-4.69B,NaN,NaN
1222,20:01,GBP,NaN,BRC Shop Price Index (YoY),8.0%,NaN,7.3%,NaN,NaN
1223,20:30,KRW,NaN,S&P Global South Korea Manufacturing PMI (Jan),48.5,NaN,48.5,NaN,NaN
1224,20:30,JPY,NaN,au Jibun Bank Japan Manufacturing PMI (Jan),48.9,48.9,48.9,NaN,NaN


In [23]:
df_filtered = df_raw[~df_raw['Cur.'].isna()] # Exclude holidays
df_filtered.loc[:, 'Date'] = df_filtered['Imp.'].ffill() # Get dates for all rows
df_filtered = df_filtered[df_filtered['Imp.'].isna()] # Remove rows with dates

C:\Users\lukas\AppData\Local\Temp\ipykernel_13224\3107507473.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'Date'] = df_filtered['Imp.'].ffill() # Get dates for all rows


In [24]:
flags = driver.find_elements(
    By.XPATH, '//*[@class="left flagCur noWrap"]/span'
)

countries = []
for flag in flags:
    countries.append(flag.get_attribute('title'))

df_filtered['Country'] = countries

In [ ]:
df_filtered = df_filtered[~df_filtered['Actual'].isna()] # Get only indicators
df_filtered.rename(columns={'Cur.': 'Currency'}, inplace=True)
df_filtered['DateTime'] = pd.to_datetime(df_filtered['Date'] + ' ' + df_filtered['Time'])

df = df_filtered[['DateTime', 'Country', 'Currency', 'Event',
                  'Actual', 'Forecast', 'Previous']]
df

In [32]:
df.to_csv('economic_calendar.csv')